In [1]:
import os
from kafka import KafkaProducer
import pandas as pd
import json
import time
from dotenv import load_dotenv
import heapq

from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, window, lit, when, expr
from pyspark.sql.types import StructType, StringType, TimestampType, IntegerType, DoubleType


load_dotenv(dotenv_path=os.path.expanduser("/home/debian/.bashrc"), override=True)
# os.environ["SHELL"] = "/usr/bin/bash"
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/opt/spark"


master_url = os.environ.get("SPARK_MASTER_URL")
kafka_bootstrap_servers = os.environ.get("KAFKA_BROKERS")


trip_schema = StructType() \
    .add("trip_id", StringType()) \
    .add("timestamp", TimestampType()) \
    .add("event", StringType()) \
    .add("PULocationID", IntegerType()) \
    .add("DOLocationID", IntegerType()) \
    .add("passenger_count", DoubleType())

trip_end_schema = StructType() \
    .add("trip_id", StringType()) \
    .add("timestamp", TimestampType()) \
    .add("event", StringType()) \
    .add("PULocationID", IntegerType()) \
    .add("DOLocationID", IntegerType()) \
    .add("total_amount", DoubleType()) 

print("done")

python-dotenv could not parse statement starting at line 6
python-dotenv could not parse statement starting at line 7
python-dotenv could not parse statement starting at line 8
python-dotenv could not parse statement starting at line 16
python-dotenv could not parse statement starting at line 24
python-dotenv could not parse statement starting at line 34
python-dotenv could not parse statement starting at line 39
python-dotenv could not parse statement starting at line 40
python-dotenv could not parse statement starting at line 47
python-dotenv could not parse statement starting at line 49
python-dotenv could not parse statement starting at line 58
python-dotenv could not parse statement starting at line 64
python-dotenv could not parse statement starting at line 67
python-dotenv could not parse statement starting at line 76
python-dotenv could not parse statement starting at line 77
python-dotenv could not parse statement starting at line 78
python-dotenv could not parse statement sta

done


In [2]:
print(master_url)
try:
    spark.stop()
except:
    pass
spark = SparkSession.builder \
    .appName("TaxiTripStream2") \
    .master(master_url) \
    .config("spark.ui.host", "0.0.0.0") \
    .config("spark.ui.port", "4040") \
    .config("spark.executor.memory", "2G") \
    .config("spark.driver.memory", "2G") \
    .config("spark.cores.max", "4") \
    .config("spark.jars.packages",
    "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1")\
    .config("spark.hadoop.fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem") \
    .config("spark.hadoop.fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS") \
    .getOrCreate()

spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")
 # .config("spark.driver.host", "10.186.0.4") \
 #.config("spark.driver.bindAddress","34.116.218.2")  \
#.config("spark.driver.port", "7078") \

spark://34.118.16.181:7077
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-064e2622-de69-4ed6-8171-c0a90aa16564;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.1 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 1215ms :: artifacts dl 49ms
	:: modules in u

In [ ]:
df_raw = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", "trips-start") \
    .option("startingOffsets", "earliest") \
    .load()

df_parsed = df_raw.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), trip_schema).alias("data")) \
    .select("data.*")

query = df_parsed.writeStream \
    .option("truncate", False) \
    .outputMode("update") \
    .format("console") \
    .foreachBatch(lambda df, b: df.show()) \
    .option("checkpointLocation", "/tmp/checkpoints/trips-start") \
    .start()

query.awaitTermination(20)
query.stop()

In [6]:


data = [("Alice", 25), ("Bob", 30), ("Charlie", 35)]
df = spark.createDataFrame(data, ["name", "age"])

# Run a simple transformation and action
df_filtered = df.filter(df.age > 28)
df_filtered.show()


[Stage 45:======================================>                   (2 + 1) / 3]

+-------+---+
|   name|age|
+-------+---+
|    Bob| 30|
|Charlie| 35|
+-------+---+



In [3]:
# Trips-start stream
start_df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", "trips-start") \
    .option("startingOffsets", "earliest") \
    .load()

start_parsed = start_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), trip_schema).alias("data")) \
    .select("data.*")

# Trips-end stream
end_df = spark.readStream.format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", "trips-end") \
    .option("startingOffsets", "earliest") \
    .load()

end_parsed = end_df.selectExpr("CAST(value AS STRING)") \
    .select(from_json(col("value"), trip_end_schema).alias("data")) \
    .select("data.*")

# Write to console
start_query = start_parsed.writeStream \
    .format("console") \
    .outputMode("append") \
    .option("truncate", False) 
    # .start()

end_query = end_parsed.writeStream \
    .format("console") \
    .outputMode("append") \
    .option("truncate", False) 
    # .start()

# Wait for termination (adjust duration or remove for indefinite)
# start_query.start().awaitTermination(10)
# end_query.awaitTermination(10)

# start_query.stop()
# end_query.stop()

In [ ]:
start_stamped = start_parsed.withColumnRenamed("timestamp", "start_timestamp").withWatermark("start_timestamp", "10 minutes").drop("event")
end_stamped = end_parsed.withWatermark("timestamp", "10 minutes").drop("PULocationID").drop("DOLocationID").drop("event")

all_trips = start_stamped.join(end_stamped,on=[
        start_stamped["trip_id"] == end_stamped["trip_id"],
        end_stamped["timestamp"] >= start_stamped["start_timestamp"],
        end_stamped["timestamp"] <= start_stamped["start_timestamp"] + expr("interval 1 hour")
    ],how="left")

all_trips = all_trips.withColumn( "is_finished", when(col("total_amount").isNotNull(), True).otherwise(False) )

finished_trips = all_trips.filter(col("is_finished") == True)
trip_counts = finished_trips.groupBy(
        window(col("timestamp"), "1 hour"),
        col("PULocationID"),
        col("DOLocationID")
    ) \
    .count()


def process_batch(df, batch_id):
    df.orderBy(col("PULocationID").desc()).show(n=100,truncate=False)    
query = trip_counts.writeStream \
    .outputMode("append") \
    .format("console") \
    .foreachBatch(process_batch) \
    .option("truncate", False) \
    .option("checkpointLocation", "tmp/checkpoints/trip_counts2") \
    .option("numRows", 100) \
    .start()

query.awaitTermination(10)

In [4]:
# finished_trips = all_trips.filter(col("is_finished") == True)
trip_counts = end_parsed.withWatermark("timestamp", "10 minutes").groupBy(
        window(col("timestamp"), "1 hour"),
        col("PULocationID"),
        col("DOLocationID")
    ) \
    .count()


def process_batch(df, batch_id):
    df.orderBy(col("PULocationID").desc()).show(n=100,truncate=False)    
query = trip_counts.writeStream \
    .outputMode("update") \
    .format("console") \
    .foreachBatch(process_batch) \
    .option("truncate", False) \
    .option("numRows", 100) \
    .start()
# .option("checkpointLocation", "gs://zad2-461913-spark-checkpoints/tmp/check/trip_ct") \
query.awaitTermination(10)

25/06/07 17:39:33 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-c6b7191f-3643-4787-a174-d309dc703a9e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/06/07 17:39:33 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/06/07 17:39:34 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.
[Stage 0:>                                                          (0 + 1) / 1]

False

+------------------------------------------+------------+------------+-----+
|window                                    |PULocationID|DOLocationID|count|
+------------------------------------------+------------+------------+-----+
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|264         |125         |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|264         |264         |3    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |137         |2    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |74          |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |246         |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |237         |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |158         |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |163         |2    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |48          |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |79          |1    |

25/06/07 17:40:17 WARN TaskSetManager: Lost task 149.0 in stage 3.0 (TID 354) (10.186.0.7 executor 1): java.lang.IllegalStateException: Error reading delta file file:/tmp/temporary-c6b7191f-3643-4787-a174-d309dc703a9e/state/0/149/1.delta of HDFSStateStoreProvider[id = (op=0,part=149),dir = file:/tmp/temporary-c6b7191f-3643-4787-a174-d309dc703a9e/state/0/149]: file:/tmp/temporary-c6b7191f-3643-4787-a174-d309dc703a9e/state/0/149/1.delta does not exist
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider.updateFromDeltaFile(HDFSBackedStateStoreProvider.scala:464)
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider.$anonfun$loadMap$4(HDFSBackedStateStoreProvider.scala:420)
	at scala.runtime.java8.JFunction1$mcVJ$sp.apply(JFunction1$mcVJ$sp.java:23)
	at scala.collection.immutable.NumericRange.foreach(NumericRange.scala:75)
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider.$anonfun$loadMap$2(HDFSBackedStateS

+------+------------+------------+-----+
|window|PULocationID|DOLocationID|count|
+------+------------+------------+-----+
+------+------------+------------+-----+



...

25/06/07 17:41:41 WARN TaskSetManager: Lost task 185.0 in stage 19.0 (TID 2011) (10.186.0.7 executor 1): java.lang.IllegalStateException: Error reading delta file file:/tmp/temporary-c6b7191f-3643-4787-a174-d309dc703a9e/state/0/185/1.delta of HDFSStateStoreProvider[id = (op=0,part=185),dir = file:/tmp/temporary-c6b7191f-3643-4787-a174-d309dc703a9e/state/0/185]: file:/tmp/temporary-c6b7191f-3643-4787-a174-d309dc703a9e/state/0/185/1.delta does not exist
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider.updateFromDeltaFile(HDFSBackedStateStoreProvider.scala:464)
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider.$anonfun$loadMap$4(HDFSBackedStateStoreProvider.scala:420)
	at scala.runtime.java8.JFunction1$mcVJ$sp.apply(JFunction1$mcVJ$sp.java:23)
	at scala.collection.immutable.NumericRange.foreach(NumericRange.scala:75)
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider.$anonfun$loadMap$2(HDFSBackedStat

+------------------------------------------+------------+------------+-----+
|window                                    |PULocationID|DOLocationID|count|
+------------------------------------------+------------+------------+-----+
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|265         |265         |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|264         |125         |2    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|264         |264         |7    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |170         |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |137         |4    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |246         |2    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |158         |2    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |163         |4    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |224         |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |48          |3    |

25/06/07 17:41:51 WARN TaskSetManager: Lost task 152.0 in stage 21.0 (TID 2185) (10.186.0.7 executor 1): java.lang.IllegalStateException: Error reading delta file file:/tmp/temporary-c6b7191f-3643-4787-a174-d309dc703a9e/state/0/152/1.delta of HDFSStateStoreProvider[id = (op=0,part=152),dir = file:/tmp/temporary-c6b7191f-3643-4787-a174-d309dc703a9e/state/0/152]: file:/tmp/temporary-c6b7191f-3643-4787-a174-d309dc703a9e/state/0/152/1.delta does not exist
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider.updateFromDeltaFile(HDFSBackedStateStoreProvider.scala:464)
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider.$anonfun$loadMap$4(HDFSBackedStateStoreProvider.scala:420)
	at scala.runtime.java8.JFunction1$mcVJ$sp.apply(JFunction1$mcVJ$sp.java:23)
	at scala.collection.immutable.NumericRange.foreach(NumericRange.scala:75)
	at org.apache.spark.sql.execution.streaming.state.HDFSBackedStateStoreProvider.$anonfun$loadMap$2(HDFSBackedStat

+------------------------------------------+------------+------------+-----+
|window                                    |PULocationID|DOLocationID|count|
+------------------------------------------+------------+------------+-----+
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|264         |264         |11   |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|264         |88          |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |90          |2    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |263         |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |163         |5    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |249         |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |186         |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |238         |5    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |145         |1    |
|{2025-02-01 00:00:00, 2025-02-01 01:00:00}|263         |229         |7    |

[Stage 23:=============>                                         (48 + 2) / 200]

In [9]:
jars = spark.sparkContext._jsc.sc().listJars()
# for jar in jars:
print(jars)

Vector(https://storage.googleapis.com/hadoop-lib/gcs/gcs-connector-hadoop3-latest.jar)
